## Pre-processing and Data Exploration
Lets get an idea of the shape of our dataset, it's attributes, and it's null-count. Since the dataset is so large, we will select a subset of the data to focus on. 

In [154]:
import pandas as pd
import numpy as np

# read dataset
df = pd.read_csv('data/covid19.csv')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204025 entries, 0 to 204024
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    204025 non-null  object 
 1   continent                                   192210 non-null  object 
 2   location                                    204025 non-null  object 
 3   date                                        204025 non-null  object 
 4   total_cases                                 195952 non-null  float64
 5   new_cases                                   195707 non-null  float64
 6   new_cases_smoothed                          194530 non-null  float64
 7   total_deaths                                177222 non-null  float64
 8   new_deaths                                  177199 non-null  float64
 9   new_deaths_smoothed                         176035 non-null  float64
 

<a id='info'></a>

In [155]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.125,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [156]:
df.shape

(204025, 67)

### Duplicates
Check for duplicate rows and then use a nested for loop to check for duplicate columns as well. We find no duplicates, however are job is not done yet. There is a possibility that there exists 1:1 correlates in our set of features.

In [157]:
# no duplicate rows
df_dupe = df.duplicated(keep='first')
duplicate_rows = [i for i in df_dupe.index if df_dupe[i]==True]
duplicate_rows

# duplicate columns
def duplicateCols(df):
    duplicateColumnNames = []
    # iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # select column at xth index.
        col = df.iloc[:, x]
        # list of dupe columns
        lst1 = []
        # iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # select column at yth index.
            otherCol = df.iloc[:, y]
            # check if two columns at x & y index are equal
            if col.equals(otherCol):
                # append columns duplicate of column x to a list
                lst1.append(df.columns.values[y])
        # check to see if duplicate columns identified
        for i in lst1:
            # discard list if column has already been identified as a duplicate
            if not any(i in s for s in duplicateColumnNames):
                # if duplicate columns exist add column to duplicate group
                lst1.append(df.columns.values[x])
                duplicateColumnNames.append(lst1)
    return duplicateColumnNames

# no duplicate columns
covid_duplicates = duplicateCols(df)
covid_duplicates

[]

### Condensing our Dataset
To narrow down the dataset, we selected records for countries that will provide us a more diverse range of values for each feature. For example, we picked countries from different parts of the world, such as Asia, North America, Europe, Africa, the Middle East; countries with high and low GDP; countries with small and large population sizes etc. We acknowledge that in the real world, picking a sample like this would introduce selection bias and the sampling method would be more random. However, in this project, we make the assumption that our subset of data is reflective of the global population.

In [158]:
# select out countries
df = df.loc[(df['continent'] == 'Europe') | 
            (df['location'] == 'Canada') | 
            (df['location'] == 'United States of America') |
            (df['location'] == 'Australia')
            ]

microstates = ['San Marino', 'Faeroe Islands', 'Isle of Man', 
               'Guernsey', 'Gibraltar', 'Jersey', 'Vatican', 
               'Liechtenstein', 'Andorra', 'Kosovo', 'Monaco',
               'Belarus', 'Russia', 'North Macedonia', 'Montenegro']

# drop microstates 
df = df[~df['location'].isin(microstates)]

# check which countries we have and put in a list, then drop some unneeded
# columns
countries_lst = df['location'].unique()
countries_lst

array(['Albania', 'Australia', 'Austria', 'Belgium',
       'Bosnia and Herzegovina', 'Bulgaria', 'Canada', 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova',
       'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'Ukraine', 'United Kingdom'], dtype=object)

In [159]:
len(countries_lst)

38

Create a dictionary indicating the number of records for each country in our transformed dataset

In [160]:
# find # of records for each country
countries_records_dict = {}
for i in countries_lst:
    # key --> country, value --> # of records for that country 
    countries_records_dict[i] = len(df[(df['location'] == i)])

countries_records_dict

{'Albania': 883,
 'Australia': 913,
 'Austria': 883,
 'Belgium': 904,
 'Bosnia and Herzegovina': 874,
 'Bulgaria': 871,
 'Canada': 916,
 'Croatia': 883,
 'Cyprus': 871,
 'Czechia': 878,
 'Denmark': 906,
 'Estonia': 905,
 'Finland': 910,
 'France': 915,
 'Germany': 912,
 'Greece': 885,
 'Hungary': 876,
 'Iceland': 880,
 'Ireland': 879,
 'Italy': 908,
 'Latvia': 882,
 'Lithuania': 860,
 'Luxembourg': 884,
 'Malta': 874,
 'Moldova': 871,
 'Netherlands': 881,
 'Norway': 883,
 'Poland': 875,
 'Portugal': 878,
 'Romania': 882,
 'Serbia': 882,
 'Slovakia': 873,
 'Slovenia': 906,
 'Spain': 907,
 'Sweden': 907,
 'Switzerland': 883,
 'Ukraine': 876,
 'United Kingdom': 908}

### Feature Selection
Focus in on a 1 year range (2021-2022) of the data. We do not want to have unequal amounts of months in our range such as a data range with winter months appearing twice and rest of seasons appearing once. We assume winter months to have less cases/deaths compared to other months because of barriers to interaction like weather. We also want to select a range for which we can assume there is sufficient data -- in 2020 countries were not taking COVID-19 as seriously as they were in 2021, more resources are being dedicated to fighting the pandemic and recording data related to it.


In [161]:
# select 1 year range from 2021-2022
df = df.loc[(df['date'] >= '2021-01-01') & (df['date'] < '2022-01-01')]

# drop date column and other wholly irelevant columns we no longer need
df.drop(['date', 'iso_code', 'continent', 'tests_units'], axis=1, inplace=True)

In our [initial exploration](#info) we noticed that there are many columns titled 'x_per_n' -- where n is some power of ten -- or '_smoothed'. This is fine if their exists no corresponding raw column in our dataset, but this is not the case. We will remove these columns if they exist as they are essentially duplicates and we will keep only per_n or smoothed columns that do not correlate with another raw column:

In [162]:
# find all columns ending with 'per_n'
per_columns = [col for col in df.columns if col.endswith(('per_million', 'per_thousand',
                                                          'per_hundred'))]
# find all columns ending with 'smoothed'                                                    
smoothed_columns =[col for col in df.columns if col.endswith('smoothed')]

In [163]:
per_columns

['total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'icu_patients_per_million',
 'hosp_patients_per_million',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions_per_million',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed_per_thousand',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_hundred',
 'new_vaccinations_smoothed_per_million',
 'new_people_vaccinated_smoothed_per_hundred',
 'hospital_beds_per_thousand',
 'excess_mortality_cumulative_per_million']

In [164]:
smoothed_columns

['new_cases_smoothed',
 'new_deaths_smoothed',
 'new_tests_smoothed',
 'new_vaccinations_smoothed',
 'new_people_vaccinated_smoothed']

In [165]:
# find all corresponding columns if they exist
corr_per_columns = [col.split('_per')[0] for col in per_columns if col.split('_per')[0] in df.columns]
corr_smoothed_columns = [col.split('_smoothed')[0] for col in smoothed_columns if col.split('_smoothed')[0] in df.columns]

In [166]:
corr_per_columns

['total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'total_tests',
 'new_tests',
 'new_tests_smoothed',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations_smoothed',
 'new_people_vaccinated_smoothed',
 'excess_mortality_cumulative']

In [167]:
corr_smoothed_columns

['new_cases', 'new_deaths', 'new_tests', 'new_vaccinations']

In [168]:
# find which per_n and smoothed columns without corresponding dupes -- columns to keep
diff_columns1 = [x for x in per_columns if x.split('_per')[0] not in corr_per_columns]
diff_columns2 = [x for x in smoothed_columns if x.split('_smoothed')[0] not in corr_smoothed_columns]

In [169]:
diff_columns1

['hospital_beds_per_thousand']

In [170]:
diff_columns2

['new_people_vaccinated_smoothed']

We remove columns that are essentially rescaled duplicates/correlates of another column: 

In [171]:
# remove all the per_n and smoothed columns that have a corresponding raw column
per_columns.remove('hospital_beds_per_thousand')
smoothed_columns.remove('new_people_vaccinated_smoothed')
per_columns.extend(smoothed_columns)

df.drop(per_columns, axis=1, inplace=True)

# return list of resulting columns
len(df.columns)
df.columns

Index(['location', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
       'reproduction_rate', 'icu_patients', 'hosp_patients',
       'weekly_icu_admissions', 'weekly_hosp_admissions', 'total_tests',
       'new_tests', 'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_people_vaccinated_smoothed',
       'stringency_index', 'population', 'population_density', 'median_age',
       'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index',
       'excess_mortality_cumulative_absolute', 'excess_mortality_cumulative',
       'excess_mortality'],
      dtype='object')

### Dealing with Null Values

We created a dataframe that gives us a quick glimpse of the amount of records countaining a null in each column by country. We decided to remove columns where null count totaled 60% or more and reran the DataFrame to get an idea of which cells will need to be filled. We chose 60% because our input variables would be severely limited had we went with 50%.

Null values can severely hurt our models by biasing estimates of paramters. We replaced remaining null values in each column with the median value as it is less resistant to outliers (if their truly are any) than the mean.

In [172]:
# function for creating a dataframe showing us nulls
def nans(df, countries):
    countries_nans_dict = {}
    for country in countries:
        # create a new list for each country in the dict
        column_nans = []
        # iterate over dataframe columns
        for col in df.columns:
            # append to list sum of null values in each column for current country
            column_nans.append(df.loc[df['location'] == country, [col]].isnull().values.ravel().sum())
            # assign list of sum of nulls in each column as value for country key
            countries_nans_dict[country] = column_nans

    df_nans = pd.DataFrame(data=list(countries_nans_dict.values()), 
                           index=countries_nans_dict.keys(),
                           columns=df.columns)
    # return dataframe containing only columns with at least one null value for any country
    df_nans_dropped = df_nans.loc[:, df_nans.any()]
    return df_nans_dropped

pd.set_option('display.max_columns', None)
nans(df, countries_lst)

,new_cases,new_deaths,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,total_tests,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_people_vaccinated_smoothed,aged_70_older,extreme_poverty,handwashing_facilities,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
Albania,0,0,365,365,365,365,17,17,0,0,138,191,201,336,200,10,0,0,365,353,353,353
Australia,0,2,0,0,365,365,0,0,0,175,54,56,59,282,56,52,0,0,365,313,313,313
Austria,0,0,0,0,365,365,2,3,31,31,313,313,314,315,365,8,0,0,365,313,313,313
Belgium,0,0,0,0,365,0,0,0,0,0,0,0,0,251,0,0,0,0,365,313,313,313
Bosnia and Herzegovina,0,1,365,365,365,365,113,166,0,1,346,345,347,364,363,42,0,0,0,353,353,353
Bulgaria,0,0,0,0,365,365,132,192,0,0,20,150,35,345,26,0,0,0,365,313,313,313
Canada,0,1,0,0,365,365,0,0,0,0,0,1,0,41,0,0,0,0,365,313,313,313
Croatia,0,0,365,0,365,313,96,148,0,0,88,93,154,365,141,0,0,0,365,313,313,313
Cyprus,0,0,0,0,313,313,0,0,0,0,223,224,226,331,304,6,0,365,365,313,313,313
Czechia,2,5,0,0,0,0,0,0,0,0,0,0,0,162,0,0,0,365,365,313,313,313


In [173]:
# calculate percent missing NaN
def percent_missing(df):
    a = df.isnull().sum() * 100 / len(df)
    return a
percent_missing(df)

location                                 0.000000
total_cases                              0.000000
new_cases                                0.086518
total_deaths                             0.000000
new_deaths                               0.194665
reproduction_rate                        0.000000
icu_patients                            35.955299
hosp_patients                           19.055516
weekly_icu_admissions                   82.941601
weekly_hosp_admissions                  69.625090
total_tests                             13.215573
new_tests                               13.417448
positive_rate                            5.133381
tests_per_case                           6.402307
total_vaccinations                      28.031723
people_vaccinated                       29.769286
people_fully_vaccinated                 31.341024
total_boosters                          66.135544
new_vaccinations                        34.722422
new_people_vaccinated_smoothed           2.249459


In [174]:
df.shape[1]

39

In [175]:
# drop columns with 50% or more NaNs
df.dropna(thresh=len(df)*0.5, axis=1, inplace=True)

In [176]:
df.shape[1]

32

In [177]:
# check nans again 
pd.set_option('display.max_columns', None)

nans(df, countries_lst)
#and/or
percent_missing(df)

location                           0.000000
total_cases                        0.000000
new_cases                          0.086518
total_deaths                       0.000000
new_deaths                         0.194665
reproduction_rate                  0.000000
icu_patients                      35.955299
hosp_patients                     19.055516
total_tests                       13.215573
new_tests                         13.417448
positive_rate                      5.133381
tests_per_case                     6.402307
total_vaccinations                28.031723
people_vaccinated                 29.769286
people_fully_vaccinated           31.341024
new_vaccinations                  34.722422
new_people_vaccinated_smoothed     2.249459
stringency_index                   0.000000
population                         0.000000
population_density                 0.000000
median_age                         0.000000
aged_65_older                      0.000000
aged_70_older                   

In [178]:
def fill(df):    
    for col in df.select_dtypes(include=['float']).columns.tolist():
        # fill nulls for each country with the median value for each column
        for country in countries_lst:
            if df.loc[df['location'] == country, col].isnull().sum() == 365:
                df.loc[df['location'] == country, col] = df.loc[df['location'] == country, col].fillna(value=df[col].median())
            else:
                med = float(df.loc[df['location'] == country, col].median())
                df.loc[df['location'] == country, col] = df.loc[df['location'] == country, col].fillna(value=med)
    return df
fill(df)

,location,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,total_tests,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_people_vaccinated_smoothed,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
2090,Albania,58316.0,0.0,1181.0,0.0,0.96,209.0,742.0,249384.0,1858.0,0.1940,5.2,1021408.0,761467.5,601104.0,8134.0,2661.0,56.48,2854710.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,2.89,78.57,0.795
2091,Albania,58991.0,675.0,1190.0,9.0,0.98,209.0,742.0,250409.0,1025.0,0.2268,4.4,1021408.0,761467.5,601104.0,8134.0,2661.0,56.48,2854710.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,2.89,78.57,0.795
2092,Albania,59438.0,447.0,1193.0,3.0,1.00,209.0,742.0,252718.0,2309.0,0.2209,4.5,1021408.0,761467.5,601104.0,8134.0,2661.0,56.48,2854710.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,2.89,78.57,0.795
2093,Albania,59623.0,185.0,1199.0,6.0,1.01,209.0,742.0,253999.0,1281.0,0.2150,4.7,1021408.0,761467.5,601104.0,8134.0,2661.0,56.48,2854710.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,2.89,78.57,0.795
2094,Albania,60283.0,660.0,1210.0,11.0,1.03,209.0,742.0,257423.0,3424.0,0.2101,4.8,1021408.0,761467.5,601104.0,8134.0,2661.0,56.48,2854710.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,2.89,78.57,0.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192031,United Kingdom,12263927.0,107771.0,148125.0,143.0,1.32,870.0,9913.0,386752248.0,1388049.0,0.0742,13.5,131743125.0,51681547.0,47294774.0,315382.0,20531.0,48.61,67281040.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.932
192032,United Kingdom,12402073.0,138146.0,148144.0,19.0,1.32,851.0,11075.0,388303628.0,1551380.0,0.0789,12.7,132121605.0,51705406.0,47324308.0,378480.0,18232.0,48.61,67281040.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.932
192033,United Kingdom,12584985.0,182912.0,148202.0,58.0,1.32,868.0,12123.0,389886084.0,1582456.0,0.0874,11.4,132633271.0,51738013.0,47368074.0,511666.0,17275.0,48.61,67281040.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.932
192034,United Kingdom,12774047.0,189062.0,148534.0,332.0,1.31,875.0,13314.0,391519809.0,1633725.0,0.0948,10.6,133108303.0,51771384.0,47412181.0,475032.0,17384.0,48.61,67281040.0,272.898,40.8,18.517,12.527,39753.244,0.2,122.137,4.28,20.0,24.7,2.54,81.32,0.932


In [179]:
# remove location, no longer needed
df.drop(['location'], axis=1, inplace=True)

df.columns

Index(['total_cases', 'new_cases', 'total_deaths', 'new_deaths',
       'reproduction_rate', 'icu_patients', 'hosp_patients', 'total_tests',
       'new_tests', 'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_people_vaccinated_smoothed', 'stringency_index', 'population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index'],
      dtype='object')

### Detecting outliers
The best way to find outliers is too see which values fall outside the lower and upper bounds of the interquartile range. We create a dataframe to see portion of values in a column that fit this criteria. However, we decide not to remove or alter these outliers just yet -- these 'outliers' could be motivated by peaks of waves of covid (e.g., delta variant) which we know result in huge spikes of cases, deaths, vaccinations, and other variables in this dataset. They could also represent natural differences in features between countries covered in our dataset.

In [180]:
def outliers(df):
    UB_lst = []
    LB_lst = [] 
    for col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        LB = Q1 - 1.5*IQR
        UB = Q3 + 1.5*IQR
        #df.loc[df[col] > UB,col] = UB
        UB_lst.append(df[df[col] > UB].shape[0]/ df[col].count())
        #df.loc[df[col] < LB,col] = LB
        LB_lst.append(df[df[col] < LB].shape[0]/ df[col].count())
    return UB_lst, LB_lst
outlier_df = pd.DataFrame(data=outliers(df), columns=df.columns, index=['>UB', '<LB'])
outlier_df

,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,total_tests,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_people_vaccinated_smoothed,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
>UB,0.122927,0.122927,0.149603,0.122422,0.009805,0.139798,0.14535,0.11478,0.140303,0.056597,0.113194,0.135472,0.143259,0.128335,0.141312,0.108652,0.0,0.131579,0.105263,0.0,0.000000,0.0,0.026316,0.105263,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
<LB,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.026316,0.0,0.000000,0.000000,0.0,0.0,0.052632,0.0,0.0,0.0,0.0


For good measure we compare the mean and median for each columns, nothing appears to be particularily off and we decide not to touch the outliers:

In [181]:
def mean_median(df):
    mm_lst = []
    for col in df.columns:
        mm_lst.append(df[col].agg(['mean', 'median']))
    return mm_lst
mean_median_df = pd.DataFrame(data = mean_median(df), columns = ['mean', 'median'])
mean_median_df

,mean,median
total_cases,1.194630e+06,5.238410e+05
new_cases,4.267791e+03,9.890000e+02
total_deaths,2.480839e+04,1.063500e+04
new_deaths,5.360552e+01,1.100000e+01
reproduction_rate,1.043760e+00,1.030000e+00
icu_patients,4.439689e+02,2.090000e+02
hosp_patients,2.592333e+03,7.420000e+02
total_tests,2.339889e+07,8.086934e+06
new_tests,8.866889e+04,2.241300e+04
positive_rate,7.608975e-02,4.820000e-02


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13870 entries, 2090 to 192035
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   total_cases                     13870 non-null  float64
 1   new_cases                       13870 non-null  float64
 2   total_deaths                    13870 non-null  float64
 3   new_deaths                      13870 non-null  float64
 4   reproduction_rate               13870 non-null  float64
 5   icu_patients                    13870 non-null  float64
 6   hosp_patients                   13870 non-null  float64
 7   total_tests                     13870 non-null  float64
 8   new_tests                       13870 non-null  float64
 9   positive_rate                   13870 non-null  float64
 10  tests_per_case                  13870 non-null  float64
 11  total_vaccinations              13870 non-null  float64
 12  people_vaccinated           

In [183]:
%store df

Stored 'df' (DataFrame)
